# 機械学習によるうつ病診断のためのデータ分析

## 【背景・目的】
　精神疾患は現代社会の主要な課題であり、うつ病は特に深刻な問題となっている。厚生労働省の患者調査によれば、令和2年には約503.1千人が精神及び行動の障害と診断された[1]。精神疾患の診断は難しく、現行の基準は主観的で信頼性が低いため、補助となる指標が必要である。本研究では、精神疾患の診断や治療を補助できる機械学習技術の開発に向けたデータ分析を行う。今回は特に精神疾患の中でも社会課題となっているうつ病に着目し、音声データや脳波(EEG)などを活用して、データ分析によってうつ病診断の精度向上を目指す。これにより、医療現場だけでなく電話相談などでも音声によるうつ病の簡易診断が可能になると考えられている。

## 【実験】
　本研究では、精神疾患解析のためのマルチモーダルなオープンデータセットMulti-modal Open Dataset for Mental disorder Analysis (MODMA)を活用する[2]。MODMAにはうつ病患者群とそれに対応する健常な対照群のデータが含まれている。データは脳波と中国語の音声データの2種類であり、すべて精神科医による診断を経て選ばれている。上記のオープンデータセットより、うつ病患者と健常者の音声データセットを比較することで、うつ病と診断された患者の発話における特徴の分析を行う。音声データの分析には、音楽/音響分析で広く用いられているPythonライブラリであるLibrosaを使用する。Librosaを用いると、例えばうつ病患者と健常者のそれぞれの音声データの基本周波数(F0)を計算することができる。F0の時間変化を可視化し、得られたグラフの振幅の平均値を計算することでうつ病患者と健常者の発声を比較し、発声におけるうつ病患者の特徴を確認する。
　さらに、得られたうつ病患者と健常者の音声データを利用して、Pythonの機械学習ライブラリであるscikit-learnでロジスティック回帰モデルを構築し、またその精度の改善を目指す。

## 【結果】
Pythonを活用し、各被験者から提供されている29種類の音声のF0の分析を行った。それぞれの音声から得られる最小値、最大値などの分析結果をうつ病患者群と健常者群で比較し、データの前処理やラベル付けをした。また両群のデータを訓練データとテストデータに分けた後、ロジスティック回帰モデルを構築し、そのモデルの精度を評価した。しかし訓練データの精度が0.590, 評価データの精度が0.586となり、ランダムな予測(精度0.5)と比べてさほど高くない結果となった。モデルの精度を向上させるため、新たな特徴量としてメル周波数ケプストラム係数(MFCC)とゼロ交差率(ZCR)を抽出し、モデルも男女別のものに分けた。なお、MFCCは音声信号の特徴を抽出する手法で、人間の聴覚に基づいた周波数分析を行い、ZCRは音声波形が正負間で交差する回数を表し、無声音と有声音の区別に使われる。これにより音声信号の時間領域と周波数領域の特徴を取り入れ、両群の音声をより正確に区別できるようになった。結果として、訓練データの精度が0.719, 評価データの精度が0.756と数値が向上した。
前回よりもモデルの精度が上がったものの、目標としている0.8以上の数値を得るためにロジスティック回帰よりも複雑な多層パーセプトロン(MLP)分類器を利用した。ロジスティック回帰はモデルが単純で、計算コストが低い線形モデルである一方、MLP分類器はニューラルネットワーク(NN)に基づく非線形モデルであり、より複雑で高度な予測が可能となる。実際にMLP分類器を使用したモデルの精度は訓練データが0.492, 評価データが0.532であった。
精度が下がった原因として、MLPのようなNNモデルで重要な標準化を行っていなかったことや隠れ層のサイズがデータと合っていなかった可能性が考えられる。精度を改善するため、特徴量を標準化して隠れ層のサイズを100ユニットの2層から100ユニットと50ユニットの2層に変更し、活性化関数としてReLuと最適化アルゴリズムにAdamを使用するなど、より詳細なハイパーパラメータ設定を行ったところ、訓練データの精度が1.000, 評価データの精度が0.991との結果を得た。この結果は訓練データの精度が1.0とモデルが訓練データに適合している可能性が高く、過学習の兆候が見られるが、評価データの数値も高いことから過学習の影響は小さいと思われる。

### 【コード】

In [2]:
conda install -c conda-forge librosa

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade librosa

Note: you may need to restart the kernel to use updated packages.


データセットをcsvファイルとして保存

In [9]:
import os
import pandas as pd
import librosa
from concurrent.futures import ThreadPoolExecutor
import numpy as np

data_place = '/Users/sayakoinoue/Documents/rimas/audio_lanzhou_2015 2/'
subject_codes = ['02010001', '02010002', '02010003', '02010005', '02010006', '02010008', '02010009', '02010010', '02010011', '02010012', '02010013', '02010014', '02010015', '02010018', '02010022', '02010023', '02010024', '02010025', '02010034', '02010036', '02010037', '02010039']

def process_file(subject_code, file_number):
    file_number_str = f'{file_number:02d}'
    filename = os.path.join(data_place, subject_code, f'{file_number_str}.wav')
    try:
        y, sr = librosa.load(filename)
        
        # MFCC を計算
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1) 
        mfcc_std = np.std(mfcc, axis=1)    
        
        # ZCR を計算
        zcr = librosa.feature.zero_crossing_rate(y)
        zcr_mean = np.mean(zcr)  
        
        return [subject_code, file_number_str, len(y), np.mean(y), np.std(y), np.min(y), np.quantile(y, 0.25), np.quantile(y, 0.5), np.quantile(y, 0.75), np.max(y), *mfcc_mean, *mfcc_std, zcr_mean]
    
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# ファイルの処理を並列化
with ThreadPoolExecutor(max_workers=4) as executor:
    results = [executor.submit(process_file, sc, fn) for sc in subject_codes for fn in range(1, 29 + 1)]
    results = [f.result() for f in results if f.result()]

# DataFrameに変換してcsvファイルとして保存
columns = ['subject', 'utterance', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 
           'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
           'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 
           'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'mfcc_std_13', 
           'zcr_mean']

df = pd.DataFrame(results, columns=columns)
df.to_csv('result_MDD_with_features.csv', index=False)

In [ ]:
data_place = '/Users/sayakoinoue/Documents/rimas/audio_lanzhou_2015 2/'
subject_codes = ['02030001', '02030002', '02030004', '02030005', '02030006', '02030007', '02030008', '02030009', '02030010', '02030014','02030015', '02030016', '02030017', '02020004', '02020007', '02020008', '02020010', '02020011', '02020014', '02020015', '02020016', '02020018', '02020019', '02020021',  '02020022',  '02020023', '02020025', '02020026', '02020026', '02020027']

def process_file(subject_code, file_number):
    file_number_str = f'{file_number:02d}'
    filename = os.path.join(data_place, subject_code, f'{file_number_str}.wav')
    try:
        y, sr = librosa.load(filename)
        
        # MFCC を計算
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)  
        mfcc_std = np.std(mfcc, axis=1)    
        
        # ZCR を計算
        zcr = librosa.feature.zero_crossing_rate(y)
        zcr_mean = np.mean(zcr)  
        
        return [subject_code, file_number_str, len(y), np.mean(y), np.std(y), np.min(y), np.quantile(y, 0.25), np.quantile(y, 0.5), np.quantile(y, 0.75), np.max(y), *mfcc_mean, *mfcc_std, zcr_mean]
    
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# ファイルの処理を並列化
with ThreadPoolExecutor(max_workers=4) as executor:
    results = [executor.submit(process_file, sc, fn) for sc in subject_codes for fn in range(1, 29 + 1)]
    results = [f.result() for f in results if f.result()]

# DataFrameに変換してcsvファイルとして保存
columns = ['subject', 'utterance', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 
           'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
           'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 
           'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'mfcc_std_13', 
           'zcr_mean']

df = pd.DataFrame(results, columns=columns)
df.to_csv('result_HC_with_features.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
import librosa
import numpy as np

warnings.simplefilter(action='ignore', category=FutureWarning)

# データを読み込んでラベルを割り当てる
df1 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_MDD_24July_with_features.csv").assign(label=1)
df2 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_HC_24July_with_features.csv").assign(label=0)

# データフレームを連結
df = pd.concat([df1, df2], ignore_index=True)

MFCCとZCRを取り入れた訓練データと評価データの精度

In [ ]:
from sklearn.impute import SimpleImputer

# 欠損値を平均で補完
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 性別情報を数値に変換 (M -> 1, F -> 0)
df['gender'] = df['gender'].map({'M': 1, 'F': 0})

# 特徴行列とラベルを定義
X, y = df[['count', 'gender', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 
           'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
           'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 
           'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'mfcc_std_13', 
           'zcr_mean']], df['label']

# データをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# モデルを構築してトレーニング
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# トレーニングセットで予測を行い、精度を評価
train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train Accuracy:", train_accuracy)

# テストセットで予測を行い、精度を評価
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test Accuracy:", test_accuracy)

MLP分類器を使用したモデルの訓練データと評価データの精度

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
import warnings
import librosa
import numpy as np

warnings.simplefilter(action='ignore', category=FutureWarning)

# データを読み込んでラベルを割り当てる
df1 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_MDD_24July_with_features.csv").assign(label=1)
df2 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_HC_24July_with_features.csv").assign(label=0)

# データフレームを連結する
df = pd.concat([df1, df2], ignore_index=True)

# 性別情報を数値に変換する (M -> 1, F -> 0)
df['gender'] = df['gender'].map({'M': 1, 'F': 0})

# 特徴行列とラベルを定義
X, y = df[['count', 'gender', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 
           'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
           'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 
           'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'mfcc_std_13', 
           'zcr_mean']], df['label']

# 欠損値を平均で補完
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# データをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# MLP分類機を構築してトレーニングする
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)

# トレーニングセットで予測を行い、精度を評価
train_accuracy = accuracy_score(y_train, mlp_model.predict(X_train))
print("Train Accuracy:", train_accuracy)

# テストセットで予測を行い、精度を評価
test_accuracy = accuracy_score(y_test, mlp_model.predict(X_test))
print("Test Accuracy:", test_accuracy)

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import warnings
import numpy as np

warnings.simplefilter(action='ignore', category=FutureWarning)

# データを読み込んでラベルを割り当てる
df1 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_MDD_24July_with_features.csv").assign(label=1)
df2 = pd.read_csv("/Users/sayakoinoue/Documents/rimas/result_HC_24July_with_features.csv").assign(label=0)

# データフレームを連結する
df = pd.concat([df1, df2], ignore_index=True)

# 性別情報を数値に変換する (M -> 1, F -> 0)
df['gender'] = df['gender'].map({'M': 1, 'F': 0})

# 特徴行列とラベルを定義
X, y = df[['count', 'gender', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 
           'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
           'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 
           'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'mfcc_std_13', 
           'zcr_mean']], df['label']

# 欠損値を平均で補完
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 特徴量を標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# データをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# MLP分類機を構築してトレーニング
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', 
                          alpha=0.0001, batch_size='auto', learning_rate='adaptive', 
                          max_iter=1000, random_state=42)
mlp_model.fit(X_train, y_train)

# トレーニングセットで予測を行い、精度を評価
train_accuracy = accuracy_score(y_train, mlp_model.predict(X_train))
print("Train Accuracy:", train_accuracy)

# テストセットで予測を行い、精度を評価
test_accuracy = accuracy_score(y_test, mlp_model.predict(X_test))
print("Test Accuracy:", test_accuracy)


Train Accuracy: 1.0
Test Accuracy: 0.9913793103448276


## 【今後の予定】
　うつ病患者と健常者の分類において、発声データが時間とともに変化することから音の変化やバターンを捉えるのに適している再帰型ニューラルネットワーク(RNN)で、MFCCから細かいパターンを捉えて処理するなど、発声データの分類タスクのモデル構築に挑戦する。

## 【参考文献】
[1]“令和２年（２０２０）患者調査の概況.” 厚生労働省, Sept. 2020, https://www.mhlw.go.jp/toukei/saikin/hw/kanja/20/dl/toukei.pdf. (アクセス日:2/27/2023.)
[2] Cai, H., Gao, Y., Sun, S., Li, N., Tian, F., Xiao, H., Li, J., Yang, Z., Li, X., Zhao, Q., Liu, Z., Yao, Z., Yang, M., Peng, H., Zhu, J., Zhang, X., Hu, X., & Hu, B. (2020). MODMA dataset: a Multi-modal Open Dataset for Mental-disorder Analysis. arXiv preprint arXiv:2002.09283.
[3] McFee, B., Raffel, C., Liang, D., Ellis, D. P., McVicar, M., Battenberg, E., & Nieto, O. (2015). librosa: Audio and music signal analysis in python. In Proceedings of the 14th python in science conference (Vol. 8).